In [1]:
import os, sys, json

autocog_home = os.path.realpath(f"{os.getcwd()}/../")
sys.path.append(autocog_home)

from autocog import CogArch
from autocog.arch.utility import PromptTee
from autocog.sta.syntax import Syntax

from autocog.utility.pynb import wrap_graphviz, display

In [2]:
from autocog.lm import Llama
lm = Llama(model_path="/data/models/llama-2-7b-chat.Q4_K_M.gguf", n_ctx=4096)
# from autocog.lm import RLM
# lm = RLM()

In [3]:
arch = CogArch(
    pipe=PromptTee(prefix='qna', tee=sys.stdout), lm=lm,
    syntax=Syntax(header_pre_post=('[INST]\n<<SYS>>You are an expert interacting with the world using an interactive prompting system.<</SYS>>','[/INST]'))
)

cogs = [
    arch.load('mmlu-repeat',       filepath=f"{autocog_home}/library/mmlu-exams/repeat.sta"),
    arch.load('mmlu-select',       filepath=f"{autocog_home}/library/mmlu-exams/select.sta"),
    arch.load('mmlu-select-cot',   filepath=f"{autocog_home}/library/mmlu-exams/select-cot.sta"),
    arch.load('mmlu-select-hyp',   filepath=f"{autocog_home}/library/mmlu-exams/select-hyp.sta"),
    arch.load('mmlu-select-annot', filepath=f"{autocog_home}/library/mmlu-exams/select-annot.sta"),
    arch.load('mmlu-select-iter',  filepath=f"{autocog_home}/library/mmlu-exams/select-iter.sta")
]

data = [
    {
        "topic" : "arithmetic",
        "question" : "What is 3*4+9?",
        "choices" :[ "16", "21", "39", "42" ]
    }
]

In [4]:
# TODO node prefixed by prompt name
dotstr = '\n'.join([ prompt.toGraphViz_concrete() for prompt in cogs[2].prompts.values() ])
# dotstr = cogs[0].prompts['main'].instantiate(syntax=arch.syntax, stacks={}, inputs=data[0]).toGraphViz()
display(wrap_graphviz(dotstr))

In [5]:
cogs[0].prompts['main'].concretes.keys()

dict_keys(['root@', 'topic_1_0@0', 'question_1_1@0', 'choices_1_2@0', 'choices_1_2@1', 'choices_1_2@2', 'choices_1_2@3', 'answer_1_3@0'])

In [6]:
results = [ await cog(**data[0]) for cog in cogs[2:3] ]

0: 3.3125459304012633e-270
0 > 0: > topic(text(20)): arithmetic
0 > 1: > question(text(50)): What is 3*4+9?
0 > 2: > choices(text(40))[0]: 16
0 > 3: > choices(text(40))[1]: 21
0 > 4: > choices(text(40))[2]: 39
0 > 5: > choices(text(40))[3]: 42
0 > 6: > work(thought)[0]:  work(thought)[0]:  thinking...
0 > 7: > work(thought)[1]: :  multiplying 3 and 4 gives us 
0 > 8: > work(thought)[2]: :  adding 9 to the result...
0 > 9: > work(thought)[3]: :  the answer is 16!
0 > 10: > work(thought)[4]: :  sorry, that was a mistake, the answer is
0 > 11: > work(thought)[5]: :  let me recheck... 
0 > 12: > work(thought)[6]: :  okay, I see now, the answer is 
0 > 13: > work(thought)[7]: :  great, I got it right! 
0 > 14: > work(thought)[8]: :  the answer is 21! 
0 > 15: > answer(select(choices)): 0
0 > 16: next: return
1: 7.601723628224932e-270
1 > 0: > topic(text(20)): arithmetic
1 > 1: > question(text(50)): What is 3*4+9?
1 > 2: > choices(text(40))[0]: 16
1 > 3: > choices(text(40))[1]: 21
1 > 4: > c